# Server Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import time

from datetime import date, datetime, timedelta

import csv
import json

import logging
import threading
import queue

from common import Printable, projdir

from constants import *

## Logging

Initialise the server logging.

In [2]:
def initLogging(level=logging.INFO, filename=None):
    """Initialise logging"""
    
    formatString = "%(asctime)s,%(levelname)s,%(message)s"
    dateTimeFormat = "%Y-%m-%d %H:%M:%S"

    # TODO - implement filename
    logging.basicConfig(format=formatString, datefmt=dateTimeFormat, level=level)
    
    
if __name__ == '__main__':

    # TODO - use app config for level
    initLogging(level=logging.INFO)

## Generic Server Thread and Thread Pool

See https://logiqx.github.io/sse-results/tech/motion.html for further details about the thread model.

In [3]:
class WorkerThread():
    """Generic class for server threads"""

    def __init__(self, name, stopEvent, inQueue, outQueue):
        """Simple initialisation"""

        self.name = name
        self.stopEvent = stopEvent

        self.inQueue = inQueue
        self.outQueue = outQueue

        self.thread = threading.Thread(target=self.loop)


    def start(self):
        """Start the thread"""

        logging.debug("Thread %s starting", self.name)
        self.thread.start()


    def loop(self):
        """Do the actual processing, until the stop event is detected"""

        while not self.stopEvent.is_set():
            try:
                # TODO - config driven timeout
                motion = self.inQueue.get(block=False, timeout=10)

                self.process(motion)
                
                self.outQueue.put(motion)
                
                self.inQueue.task_done()

            except queue.Empty:
                pass
            
            except Exception:
                logging.error("Exception in DetectionThread")


    def process(self, motion):
        """Default behavior suggest the actual process() method is missing"""
        
        logging.warning("Thread %s has an empty process() method", self.name)


    def join(self):
        """Join the thread, essentially waiting for it to stop"""

        self.thread.join()
        logging.debug("Thread %s finished", self.name)

In [4]:
class WorkerThreadPool(WorkerThread):
    """Generic class for a worker thread pool, potentially consisting of a manager thread and multiple worker threads"""

    def __init__(self, name, stopEvent, workerClass, workerCount, inQueue, outQueue):
        """Simple initialisation"""
        
        super().__init__(name, stopEvent, inQueue, outQueue)

        self.workers = []

        for i in range(workerCount):
            worker = workerClass(f'{name} {i}', stopEvent, inQueue, outQueue)
            self.workers.append(worker)


    def start(self):
        """Start the threads"""

        super().start()
        
        for worker in self.workers:
            worker.start()


    def loop(self):
        """There is no processing to be done in an unmanaged pool"""

        pass
    

    def join(self):
        """Join the threads, ensuring they have all stopped cleanly"""
        
        super().join()
        
        for worker in self.workers:
            worker.join()

In [5]:
class SupervisedWorkerThreadPool(WorkerThreadPool):
    """Generic class for a supervised worker thread pool, consisting of a manager thread and multiple worker threads"""
    
    def loop(self):
        """Do the actual processing, until the stop event is detected"""

        while not self.stopEvent.is_set():
            self.process()


    def process(self):
        """Default behavior suggest the actual process() method is missing"""
        
        logging.warning("Thread %s has an empty process() method", self.name)

## Detection Thread Pool

Detect new motions appearing on the network

In [6]:
class DetectionThread(WorkerThread):
    """Thread pool worker to detect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""
        
        logging.info("Thread %s detecting motion %i", self.name, motion)

In [7]:
class DetectionThreadPool(SupervisedWorkerThreadPool):
    """Thread pool manager to detect motions appearing on the network"""

    def process(self):
        """Do the actual processing"""
        
        try:
            # TODO - actual code based on address info being unavailable
            motionCount = 4
            for i in range(motionCount):
                # The manager puts motions onto the in queue for workers to consume
                # logging.info("Queueing %i...", i)
                self.inQueue.put(i)

            # TODO - sensible sleep
            self.stopEvent.wait(5)

        except Exception:
            logging.error("Exception in DetectionThreadPool")


## Motion Inspection Thread

Inspect new motions appearing on the network

In [8]:
class InspectionThread(WorkerThread):
    """Thread pool worker to inspect a motion appearing on the network"""

    def process(self, motion):
        """Do the actual inspection of a single motion"""
        
        logging.info("Thread %s inspecting motion %i", self.name, motion)

## Download Thread Pool

Download new tracks for Motions

In [9]:
class DownloadThread(WorkerThread):
    """Thread pool worker to download tracks from a motion"""

    def process(self, motion):
        """Do the actual downloads for a single motion"""
        
        logging.info("Thread %s downloading from motion %i", self.name, motion)

## Cleanup Thread Pool

Detect motions disconnecting from the network

In [10]:
class CleanupThread(WorkerThread):
    """Thread pool worker to detect a motion disappearing from the network"""

    def process(self, motion):
        """Do the actual processing of a single item"""
        
        logging.info("Thread %s checking motion %i", self.name, motion)

In [11]:
class CleanupThreadPool(SupervisedWorkerThreadPool):
    """Thread pool manager to detect motions disappearing on the network"""

    def process(self):
        """Do the actual processing"""
        
        try:
            # TODO - actual code based on address info being unavailable
            motionCount = 4
            for i in range(motionCount):
                # The manager puts motions onto the in queue for workers to consume
                # logging.info("Queueing %i...", i)
                self.inQueue.put(i)

            # TODO - sensible sleep
            self.stopEvent.wait(5)

        except Exception:
            logging.error("Exception in DisconnectionThreadPool")


## Motion Class

Stuff relating to a single Motion device

In [12]:
HOST_STATUS_UNKNOWN = 0
HOST_STATUS_READY = 1
HOST_STATUS_REFUSED = 2
HOST_STATUS_TIMEOUT = 3
HOST_STATUS_GAIERROR = 4

DEFAULT_HTTP_PORT=80

DEFAULT_PING_TIMEOUT=5
DEFAULT_JSON_TIMEOUT=10

In [13]:
class Motion():
    '''Motion GPS'''
    
    def __init__(self, identifier):
        self.identifier = identifier

        self.hostname = f'motion-{identifier}'

        self.addrInfo = None
        self.status = HOST_STATUS_UNKNOWN

        self.config = {}
        self.json = {}
        
        
    def getAddrInfo(self, port=DEFAULT_HTTP_PORT):
        """Get address information (IP address) for Motion"""

        try:
            addrInfo = socket.getaddrinfo(self.hostname, port, proto=socket.IPPROTO_TCP)[0][4]

            # Sky SR101 router will sometimes return 10.10.10.10 and should be treated the same as socket.gaierror
            if addrInfo[0] == '10.10.10.10':
                raise socket.gaierror

        # Only really expect a socket.gaierror exception, either from socket.getaddrinfo or because of 10.10.10.10
        except socket.gaierror:
            self.addrInfo = None

        # Just in case of some other (unexpected) issue!
        except Exception:
            self.addrInfo = None

        # The IP address of the Motion has been successfully identified. Yay!
        else:
            self.addrInfo = addrInfo
            
        return self.addrInfo


    def getStatus(self, timeout=DEFAULT_PING_TIMEOUT):
        """Check the status of the Motion.

        Note the use of self.addrInfo instead of using the hostname with s.connect((host, port)).

        This avoids erroneous 'gaierror: [Errno -5] No address associated with hostname' under certain circumstances."""

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()

        # Status can only be checked if address information is available
        if self.addrInfo:
            try:
                # Attempt connection to the standard HTTP socket as determined by socket.getaddrinfo()
                s = socket.socket()
                s.settimeout(timeout)
                s.connect(self.addrInfo)

                # Close the connection in a timely fashion - good practice
                s.shutdown(socket.SHUT_RDWR)
                s.close()

            # This should only happen if the IP address does not belong to a Motion
            except ConnectionRefusedError:
                self.status = HOST_STATUS_REFUSED

            # This should only happen when the Motion has been disconnected from the WiFi
            except socket.timeout:
                self.status = HOST_STATUS_TIMEOUT

            # Just in case of some other (unexpected) issue!
            except Exception:
                self.status = HOST_STATUS_UNKNOWN

            # The Motion is accepting connections. Yay!
            else:
                self.status = HOST_STATUS_READY

        # Address information is not known
        else:
            self.status = HOST_STATUS_GAIERROR

        return self.status
   

    def fetchJson(self, path, port=DEFAULT_HTTP_PORT, timeout=DEFAULT_JSON_TIMEOUT):
        """Fetch JSON from Motion"""

        result = None

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()

        # JSON can only be fetched if address information is available
        if self.addrInfo:
            try:
                # Use addrInfo to avoid unnecessary DNS lookup which can take several seconds
                url = 'http://{}/{}'.format(self.addrInfo[0], path)

                # This is a simple but effective way to open a URL in Python
                response = urlopen(url, timeout=timeout)
                txt = response.read()

                # Parse the JSON and store it in a dictionary
                result = json.loads(txt)
                self.json[path] = result

            except Exception:
                print("Failed to retrieve {}".format(url))

        return result


    def summarise(self):
        """Print summary of device"""

        #print(self.hostname)
        #print()

        info = self.json['info.json']

        #print('Username = {}'.format(info['motion']['username']))
        print('Identifier = {}'.format(info['motion']['identifier']))
        #print()

        hardware = info['motion']['hardware']
        firmware = info['motion']['firmware']

        #print('Hardware = {}'.format(hardware))
        print('Firmware = {}{}'.format(firmware, ' (latest)' if firmware == LATEST_FIRMWARE else ''))
        #print()

        # 'filesystem level' always appears to be 3

        level = info['motion']['battery']['level']
        charging = info['motion']['battery']['charging']

        print('Battery = {}%'.format(level))
        print('Battery is {}charging'.format('not ' if charging == False else ''))
        #print()

        settings = self.json['settings.json']

        print('GNSS rate = {}'.format(settings['settings']['gnss_rate']))
        print()

        # Note: 'units': {'speed': 'kn', 'distance': 'nm'} does not impact logs (mm/s and mm)
        #        'start_date' is in UTC
        #        'timezone' and 'units' are applied by the built-in webserver
        #        'mode' always appears to be 0

        #logs = fetchJson(addresses[i][0], 'logs.json', timeout=20)
        #for log in reversed(logs['logs'][-1:]):
            #print(log)
        #print()

## Main Function

Startup and shutdown of all threads

In [14]:
def dateRange(startDate, endDate):
    '''Determine range of dates'''

    dates = []

    for i in range(int((endDate - startDate).days) + 1):
        dates.append(startDate + timedelta(days = i))
        
    return dates


def readMotions():
    '''Read Motions from CSV file'''
    
    motions = {}

    year = datetime.now().year
    csvPath = os.path.join(projdir, EVENTS_DIR, str(year), CONFIG_DIR, 'motions.csv')

    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for motionUser in csvReader:
            motionId = int(motionUser['Motion ID'])

            startDate = datetime.strptime(motionUser['Start Date'], '%Y-%m-%d')
            endDate = datetime.strptime(motionUser['End Date'], '%Y-%m-%d')
            
            if motionId not in motions:
                motion = Motion(motionId)              
                motions[motionId] = motion
            else:
                motion = motions[motionId]

            for sessionDate in dateRange(startDate, endDate):
                sessionDate = sessionDate.strftime('%Y-%m-%d')

                # Note: Motion ID of zero (or negative) is a dummy record in the config
                if motionId > 0:
                    if sessionDate in motion.config:
                        logging.warning("Motion %i is assigned to multiple people on %s", motionId, sessionDate)
                    else:
                        motion.config[sessionDate] = motionUser
     
    for motionId, motion in motions.items():
        print(motionId, motion.config.keys())
    
    return motions

In [15]:
def createThreadPools(stopEvent):
    """Create the thread pools, including message queues"""

    threadPools = []

    # Detection
    
    detectionInQueue = queue.Queue()
    detectionOutQueue = queue.Queue()
    
    detectionCount = 2
    detectionThreadPool = DetectionThreadPool(
                            'detect', stopEvent, DetectionThread, detectionCount, detectionInQueue, detectionOutQueue)
    threadPools.append(detectionThreadPool)

    # Inspection
    
    inspectionInQueue = detectionOutQueue
    inspectionOutQueue = queue.Queue()
    
    inspectionCount = 2
    inspectionThreadPool = WorkerThreadPool(
                            'inspect', stopEvent, InspectionThread, inspectionCount, inspectionInQueue, inspectionOutQueue)
    threadPools.append(inspectionThreadPool)

    # Download
    
    downloadInQueue = inspectionOutQueue
    downloadOutQueue = queue.Queue()
    
    downloadCount = 2
    downloadThreadPool = WorkerThreadPool(
                            'download', stopEvent, DownloadThread, inspectionCount, downloadInQueue, downloadOutQueue)
    threadPools.append(downloadThreadPool)

    # Cleanup
    
    cleanupInQueue = queue.Queue()
    cleanupOutQueue = queue.Queue()

    cleanupCount = 2
    cleanupThreadPool = CleanupThreadPool(
                            'cleanup', stopEvent, CleanupThread, cleanupCount, cleanupInQueue, cleanupOutQueue)
    threadPools.append(cleanupThreadPool)


    return threadPools

In [16]:
def main():
    """Main function to start all of the threads"""

    logging.info("Server starting")

    # Read Motion data
    motions = readMotions()
    
    return

    # Global event so that all threads can exit gracefully
    stopEvent = threading.Event()
    
    # Start by creating the thread pools and message queues
    threadPools = createThreadPools(stopEvent)

    # Attempt to start up all of the thread pools
    threadsRunning = False

    try:
        for threadPool in threadPools:
            threadPool.start()

        threadsRunning = True
    
    except Exception:
        logging.error("Thread startup failed")
    
    # Check if thread startup was successful
    if threadsRunning:
        try:
            # Run for a maximum of 30 seconds
            time.sleep(30)
        
        # Intercept keyboard interrupts to ensure that all threads are properly stopped, prior to exit
        except KeyboardInterrupt:
            logging.info("Keyboard interrupt")

    # Raise event so that all threads can exit gracefully
    stopEvent.set()

    # Wait for all active threads to exit gracefully
    for threadPool in threadPools:
        threadPool.join()

    logging.info("Server stopped")


if __name__ == '__main__':
    main()

2022-09-30 08:49:01,INFO,Server starting


601 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21'])
611 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21'])
615 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21'])
617 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21'])
618 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21'])
619 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21'])
621 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21'])
625 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21'])
630 dict_keys(['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '20